## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt
import numpy as np
import random 
# Import API key
from config1 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_to_load = "WeatherPy_Database.csv"
city_data_df = pd.read_csv(city_data_to_load)
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Mount Gambier,AU,2021-05-25 20:55:23,-37.8333,140.7667,48.11,81,75,13.80,broken clouds
1,1,Khatanga,RU,2021-05-25 20:55:24,71.9667,102.5000,30.81,97,100,6.76,overcast clouds
2,2,New Norfolk,AU,2021-05-25 20:55:24,-42.7826,147.0587,49.75,72,47,1.01,light rain
3,3,Boende,CD,2021-05-25 20:55:24,-0.2167,20.8667,78.58,68,100,2.71,overcast clouds
4,4,Evensk,RU,2021-05-25 20:55:25,61.9500,159.2333,37.60,95,100,22.19,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
3,3,Boende,CD,2021-05-25 20:55:24,-0.2167,20.8667,78.58,68,100,2.71,overcast clouds
6,6,Kapaa,US,2021-05-25 20:55:25,22.0752,-159.3190,83.12,66,10,1.99,clear sky
8,8,Hilo,US,2021-05-25 20:51:35,19.7297,-155.0900,81.81,77,75,12.66,broken clouds
14,14,Rikitea,PF,2021-05-25 20:51:17,-23.1203,-134.9692,76.55,80,58,9.91,light rain
19,19,Avarua,CK,2021-05-25 20:53:05,-21.2078,-159.7750,82.45,83,75,8.05,broken clouds
20,20,Atuona,PF,2021-05-25 20:55:29,-9.8000,-139.0333,80.37,60,34,15.30,scattered clouds
24,24,Hithadhoo,MV,2021-05-25 20:55:30,-0.6000,73.0833,84.34,67,100,20.00,overcast clouds
28,28,Kyaikkami,MM,2021-05-25 20:55:31,16.0800,97.5675,79.83,86,100,10.31,overcast clouds
29,29,Albany,US,2021-05-25 20:55:08,42.6001,-73.9662,82.31,56,98,8.08,overcast clouds
31,31,Bolobo,CD,2021-05-25 20:55:32,-2.1667,16.2333,75.18,79,86,3.42,overcast clouds


In [34]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        189
City           189
Country        189
Date           189
Lat            189
Lng            189
Max Temp       189
Humidity       189
Cloudiness     189
Wind Speed     189
Description    189
dtype: int64

In [35]:
# https://www.codegrepper.com/code-examples/python/frameworks/django/pandas+check+if+row+is+empty
preferred_cities_df.empty == True

False

In [36]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#preferred_cities_df = preferred_cities_df.dropna(how='any')

preferred_cities_df = preferred_cities_df.dropna(how='any')

In [37]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Boende,CD,78.58,overcast clouds,-0.2167,20.8667,
6,Kapaa,US,83.12,clear sky,22.0752,-159.3190,
8,Hilo,US,81.81,broken clouds,19.7297,-155.0900,
14,Rikitea,PF,76.55,light rain,-23.1203,-134.9692,
19,Avarua,CK,82.45,broken clouds,-21.2078,-159.7750,
20,Atuona,PF,80.37,scattered clouds,-9.8000,-139.0333,
24,Hithadhoo,MV,84.34,overcast clouds,-0.6000,73.0833,
28,Kyaikkami,MM,79.83,overcast clouds,16.0800,97.5675,
29,Albany,US,82.31,overcast clouds,42.6001,-73.9662,
31,Bolobo,CD,75.18,overcast clouds,-2.1667,16.2333,


In [38]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    try: 
    # 6c. Get latitude and longitude from DataFrame
        lat = row["Lat"]
        lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
        params["location"] = f"{lat},{lng}"


    # 6d. Set up the base URL for the Google Directions API to get JSON data.
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
        hotels = requests.get(base_url, params=params).json()


    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Boende,CD,78.58,overcast clouds,-0.2167,20.8667,
6,Kapaa,US,83.12,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,Hilo,US,81.81,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
14,Rikitea,PF,76.55,light rain,-23.1203,-134.9692,Pension Maro'i
19,Avarua,CK,82.45,broken clouds,-21.2078,-159.7750,Paradise Inn
20,Atuona,PF,80.37,scattered clouds,-9.8000,-139.0333,Villa Enata
24,Hithadhoo,MV,84.34,overcast clouds,-0.6000,73.0833,Scoop Guest House
28,Kyaikkami,MM,79.83,overcast clouds,16.0800,97.5675,Happy Family Kyaikkami
29,Albany,US,82.31,overcast clouds,42.6001,-73.9662,
31,Bolobo,CD,75.18,overcast clouds,-2.1667,16.2333,Hotel de Stephen


In [46]:
hotel_df= pd.DataFrame(hotel_df)
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Boende,CD,78.58,overcast clouds,-0.2167,20.8667,
6,Kapaa,US,83.12,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,Hilo,US,81.81,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
14,Rikitea,PF,76.55,light rain,-23.1203,-134.9692,Pension Maro'i
19,Avarua,CK,82.45,broken clouds,-21.2078,-159.7750,Paradise Inn


In [47]:
# https://izziswift.com/drop-rows-containing-empty-cells-from-a-pandas-dataframe/
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

In [48]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189 entries, 3 to 670
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City         189 non-null    object 
 1   Country      189 non-null    object 
 2   Max Temp     189 non-null    float64
 3   Description  189 non-null    object 
 4   Lat          189 non-null    float64
 5   Lng          189 non-null    float64
 6   Hotel Name   172 non-null    object 
dtypes: float64(3), object(4)
memory usage: 11.8+ KB


In [49]:
hotel_df.isnull()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,False,False,False,False,False,False,True
6,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False
14,False,False,False,False,False,False,False
19,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
660,False,False,False,False,False,False,False
661,False,False,False,False,False,False,False
666,False,False,False,False,False,False,False
667,False,False,False,False,False,False,False


In [50]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df.count()


City           189
Country        189
Max Temp       189
Description    189
Lat            189
Lng            189
Hotel Name     172
dtype: int64

In [51]:
hotel_df = hotel_df.dropna(how='any')

In [52]:
hotel_df.count()



City           172
Country        172
Max Temp       172
Description    172
Lat            172
Lng            172
Hotel Name     172
dtype: int64

In [53]:
hotel_df.head(10) 

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
6,Kapaa,US,83.12,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,Hilo,US,81.81,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
14,Rikitea,PF,76.55,light rain,-23.1203,-134.9692,Pension Maro'i
19,Avarua,CK,82.45,broken clouds,-21.2078,-159.7750,Paradise Inn
20,Atuona,PF,80.37,scattered clouds,-9.8000,-139.0333,Villa Enata
24,Hithadhoo,MV,84.34,overcast clouds,-0.6000,73.0833,Scoop Guest House
28,Kyaikkami,MM,79.83,overcast clouds,16.0800,97.5675,Happy Family Kyaikkami
31,Bolobo,CD,75.18,overcast clouds,-2.1667,16.2333,Hotel de Stephen
37,Hambantota,LK,80.69,overcast clouds,6.1241,81.1185,Bungalow 63
38,Sorong,ID,77.16,overcast clouds,-0.8833,131.2500,waigo splash hotel


In [56]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [62]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Description</dt><dd>{Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]


In [63]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# https://stackoverflow.com/questions/42909099/export-interactive-google-map-as-a-static-image
# to download the image as png simply click the download button